In [19]:
import pandas as pd

In [20]:
imm = pd.read_csv('./immigration_raw.csv')
imm.head()

,Year,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Belgium,135281.0,147377.0,129477,120078,123158,146626,123702,126703,137860,150006
1,Bulgaria,NaN,NaN,14103,18570,26615,25223,21241,25597,29559,37929
2,Czechia,48317.0,27114.0,34337,30124,29897,29602,64083,51847,65910,105888
3,Denmark,52236.0,52833.0,54409,60312,68388,78492,74383,68579,64669,61384
4,Germany,404055.0,489422.0,592175,692713,884893,1571047,1029852,917109,893886,886341


In [21]:
imm = pd.melt(imm, id_vars='Year', value_vars=["20" + str(x) for x in range(10, 20)])
imm = imm.rename(columns={"Year": "country", "variable":"year", "value":"immigration"})
imm.head()

,country,year,immigration
0,Belgium,2010,135281.0
1,Bulgaria,2010,NaN
2,Czechia,2010,48317.0
3,Denmark,2010,52236.0
4,Germany,2010,404055.0


In [ ]:
imm.to_csv("./immigration.csv")